In [3]:
import mysql.connector

db = mysql.connector.connect(
  host="localhost",
  user="root",
  password="example",
  database="oarebyue_0.3"
)
cursor = db.cursor()
cursor.execute("SELECT * FROM text")
texts = cursor.fetchall()

In [4]:
textDiscourse = []
totalTexts = len(texts)
counter = 1
data_text = ''

for text in texts:
    if counter <= 500:
        cursor.execute(f'SELECT explicit_spelling FROM text_discourse where text_uuid = "{text[1]}" and explicit_spelling is not null and explicit_spelling not like "%…%" and explicit_spelling not like "%x%" and type = "word" order by word_on_tablet')
        discourse = cursor.fetchall()
        if len(discourse) > 10:
            cleanDiscourse = []
            for word in discourse:
                cleanDiscourse.append(word[0])
            textDiscourse.append({'uuid': text[1], 'words': ' '.join(cleanDiscourse)})
            data_text = data_text + ' ' + ' '.join(cleanDiscourse)
        print(f'{counter/totalTexts*100}% complete')
    counter = counter + 1

print(data_text)

0.007954816641476414% complete
0.015909633282952827% complete
0.023864449924429244% complete
0.031819266565905654% complete
0.03977408320738207% complete
0.04772889984885849% complete
0.055683716490334906% complete
0.06363853313181131% complete
0.07159334977328773% complete
0.07954816641476414% complete
0.08750298305624056% complete
0.09545779969771698% complete
0.1034126163391934% complete
0.11136743298066981% complete
0.1193222496221462% complete
0.12727706626362262% complete
0.13523188290509902% complete
0.14318669954657545% complete
0.15114151618805186% complete
0.1590963328295283% complete
0.1670511494710047% complete
0.17500596611248112% complete
0.18296078275395752% complete
0.19091559939543395% complete
0.19887041603691033% complete
0.2068252326783868% complete
0.21478004931986316% complete
0.22273486596133962% complete
0.230689682602816% complete
0.2386444992442924% complete
0.24659931588576883% complete
0.25455413252724524% complete
0.2625089491687217% complete
0.270463765810

In [5]:
import pandas as pd

df = pd.DataFrame(textDiscourse)
df['words']

0      a-na šé-ni DUMU ru-ba-um (m)ú-ki-ša a-na ar-bi...
1      KIŠIB wa-bar-tim ša ša-lá-tí-wa-ar wa-bar-tum ...
2      wa-bar-tum ša ša-lá-tí-wa-ar dí-nam i-dí-in-ma...
3      a-na um-mì-a-ni a-šur-(d)UTU-ši a-šur-i-mì-tí ...
4      um-ma šu-ḫu-bur-ma a-na pu-šu-ke-en₆ qí-bi-ma ...
                             ...                        
331    a-na pu-šu-ke-en₆ ù i-na-a qí-bi-ma um-ma šu-k...
332    a-na a-šùr-i-mì-tí en-nu-be-lim en-na-nim ù bu...
333    um-ma pu-šu-ke-en₆-ma a-na la-ma-sí qí-bi-ma m...
334    a-na a-šùr-i-mì-tí qí-bi-ma um-ma pu-šu-ke-en-...
335    um-ma PUZUR₄-a-šur-ma a-na ku-ra-ra šu-IŠTAR i...
Name: words, Length: 336, dtype: object

In [6]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.utils import to_categorical, pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Dense, GRU, Embedding
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import train_test_split

In [7]:
import re

def text_cleaner(text):
    # lower case text
    newString = text.lower()
    # newString = re.sub(r"'s\b","",newString)
    # # remove punctuations
    # newString = re.sub("[0-9]", "", newString) 

    long_words=[]
    # remove short word
    for i in newString.split():
        if len(i)>=3:                  
            long_words.append(i)
    return (" ".join(long_words)).strip()

def create_seq(text):
    length = 30
    sequences = list()
    for i in range(length, len(text)):
        # select sequence of tokens
        seq = text[i-length:i+1]
        # store
        sequences.append(seq)
    # print('Total Sequences: %d' % len(sequences))
    return sequences

# generate a sequence of characters with a language model
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict character
		yhat = model.predict_classes(encoded, verbose=0)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += char
	return in_text

In [19]:
# preprocess the text
data_new = ''
# model = Sequential()
model = tf.keras.models.load_model('model.h5')
totalTexts = len(textDiscourse)
counter = 1

data_new = text_cleaner(data_text)
# create sequences   
sequences = create_seq(data_new)
# create a character mapping index
chars = sorted(list(set(data_new)))
mapping = dict((c, i) for i, c in enumerate(chars))
tempSequences = list()
for line in sequences:
    # integer encode line
    encoded_seq = [mapping[char] for char in line]
    # store
    tempSequences.append(encoded_seq)
sequences = tempSequences
# vocabulary size
vocab = len(mapping)
sequences = np.array(sequences)
# create X and y
X, y = sequences[:,:-1], sequences[:,-1]
# one hot encode y
y = to_categorical(y, num_classes=vocab)
# create train and validation sets
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.1, random_state=42)

# print('Train shape:', X_tr.shape, 'Val shape:', X_val.shape)
# model.add(Embedding(vocab, 50, input_length=30, trainable=True))
# # model.add(LSTM(100, return_sequences=True))
# # model.add(LSTM(100))    
# model.add(GRU(150, recurrent_dropout=0.1, dropout=0.1))
# model.add(Dense(vocab, activation='softmax'))

# # compile the model
# model.compile(loss='categorical_crossentropy', metrics=['acc'], optimizer='adam')
# # fit the model
# model.fit(X_tr, y_tr, epochs=50, verbose=2, validation_data=(X_val, y_val))

In [20]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 30, 50)            2500      
                                                                 
 gru_1 (GRU)                 (None, 150)               90900     
                                                                 
 dense_1 (Dense)             (None, 50)                7550      
                                                                 
Total params: 100,950
Trainable params: 100,950
Non-trainable params: 0
_________________________________________________________________
None


In [21]:
# model.save("model2.h5")

In [22]:
import tensorflow as tf
import numpy as np

def generate_seq(model, mapping, seq_length, seed_text, n_chars):
	in_text = seed_text
	# generate a fixed number of characters
	for _ in range(n_chars):
		# encode the characters as integers
		encoded = [mapping[char] for char in in_text]
		# truncate sequences to a fixed length
		encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
		# predict character
		predictions = model.predict(encoded, verbose=0)
		# normalize probabilities
		predictions = predictions.astype('float64')
		predictions /= np.sum(predictions)
		# choose a random character based on probabilities
		probas = np.random.multinomial(1, predictions[0], 1)
		yhat = np.argmax(probas)
		# reverse map integer to character
		out_char = ''
		for char, index in mapping.items():
			if index == yhat:
				out_char = char
				break
		# append to input
		in_text += out_char
	return in_text

In [26]:
inp = 'a-na šé-ni DUMU ru-ba-um (m)ú-ki-ša a-na ar-bi₄-šu ša-ḫi-ra-tim a-sí-nu-um a-na il₅-qé-ú KIŠIB wa-bar-tim ša ša-lá-tí-wa-ar wa-bar-tum ša ša-lá-tí-wa-ar dí-nam i-dí-ma i-na GÚ ma-na URUDU SIG₅ ša ba-al-ṭù-a ú (d)UTU-e-nam a-na a-šùr-i-dí DUMU DINGIR-na-da ip-qí-du-ni KÙ.BABBAR i-na ší-im URUDU ma-lá iz-ku-a-ni ba-al-ṭù-a i-kà-na-ak-ma'
print(generate_seq(model, mapping, 30, inp.lower(), 45))

a-na šé-ni dumu ru-ba-um (m)ú-ki-ša a-na ar-bi₄-šu ša-ḫi-ra-tim a-sí-nu-um a-na il₅-qé-ú kišib wa-bar-tim ša ša-lá-tí-wa-ar wa-bar-tum ša ša-lá-tí-wa-ar dí-nam i-dí-ma i-na gú ma-na urudu sig₅ ša ba-al-ṭù-a ú (d)utu-e-nam a-na a-šùr-i-dí dumu dingir-na-da ip-qí-du-ni kù.babbar i-na ší-im urudu ma-lá iz-ku-a-ni ba-al-ṭù-a i-kà-na-ak-ma-ni zu-ku-nu ma-na an.na qá-tí a-šur-dug i-ṭí


In [ ]:
# measure head measure seal from something to Suen
# gín.ta qá-qá-da-tim gín kišib i-na a-šùr-pu-gal-ri a-na sú-en₆ 
# um-ma na-bi₄-dingir-šu-ma šu-ma ú-kà-al-ma a-ḫa-kam i-na qá-tim g
# thus proper noun  unsure    
# ta-tí-dí-šu-ma a-na-kam i-li-kam-ma ša-ma-zi-ri-im nu-ša-sú-ú-ma mì-šu-um ma-n     